<a href="https://colab.research.google.com/github/hamish-haggerty/AI-hacking/blob/master/SSL/MNIST_Val_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/hamish-haggerty/base_rbt.git#egg='base_rbt'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/hamish-haggerty/base_rbt.git to /tmp/pip-install-3j1ez8kj/base-rbt_0265a6b82c7e4e0491928f7054e0e2ca
  Running command git clone -q https://github.com/hamish-haggerty/base_rbt.git /tmp/pip-install-3j1ez8kj/base-rbt_0265a6b82c7e4e0491928f7054e0e2ca
     |████████████████████████████████| 551 kB 24.9 MB/s 
     |████████████████████████████████| 41 kB 655 kB/s 
     |████████████████████████████████| 549 kB 56.7 MB/s 
     |████████████████████████████████| 182 kB 12.1 MB/s 
  Created wheel for base-rbt: filename=base_rbt-0.0.1-py3-none-any.whl size=20565 sha256=5cc5bea4e48170f4abdf43dd88579eabadbb4d391757864f28bf703ffb7426da
  Stored in directory: /tmp/pip-ephem-wheel-cache-3yqkazhc/wheels/b3/25/be/8368facea57a956938db327bdd708e7f93ca5738de036e2c6e
Successfully built base-rbt


In [2]:
from base_rbt.base_model import * 
from base_rbt.base_lf import *
from base_rbt.base_linear import *
from base_rbt.helper import *

In [3]:
import self_supervised
import torch
from fastai.vision.all import *
from self_supervised.augmentations import *
from self_supervised.layers import *

Step 1): We need the data, and to set all the hps. 

In [4]:
#hps's
device ='cuda' if torch.cuda.is_available() else 'cpu'
cuda = (device=='cuda')
seed=20 #Will have to try with different random seeds. Two times is probably enough 
n_in=1
indim=1024 #find this by inspection, e.g. for resnet18 is 1024
size=28
ps=1024 #Will also try with other values e.g. 125, 1000 etc
bs=128 #for training BT
bs_val=20 #for training linear head
bs_test=500
ts_val=bs_val*4 #so 512, ~ 1% of 50000
ts=10000

n_epochs=300 #epochs for BT 
numfit = 200 #epochs for linear classifer

In [5]:
#Let's get MNIST data in a nicer way

path = untar_data(URLs.MNIST)
items = get_image_files(path/'training') #i.e. NOT testing!!!
items.sort()
seed_everything(seed=seed)
items=items.shuffle()
label_func=RegexLabeller(pat = r'/training/(\d)/')
labels = [label_func(i) for i in items]

items_train = items[0:ts] #train on ts samples
labels_train = labels[0:ts]
dls_train = ImageDataLoaders.from_lists(path,items_train, labels_train,bs=bs,
                                  valid_pct=0.0,num_workers=2*(device=='cuda'),device=device,item_tfms=[ToTensor()],
                                  img_cls=PILImageBW)

items_tune = items[ts:ts+3000]
labels_tune = labels[ts:ts+3000]

items_test = items[ts+3000:ts+3000+ts] #test on ts samples. 3k held out for tuning
labels_test = labels[ts+3000:ts+3000+ts]
dls_test = ImageDataLoaders.from_lists(path,items_test, labels_test,bs=bs_test,
                                  valid_pct=0.0,num_workers=2*(device=='cuda'),device=device,
                                  img_cls=PILImageBW
                                      )
def tune_set(items0,seed=42):

    seed_everything(seed=seed)
    items=items0.shuffle()
    raw=[]
    items_tune20 = []
    labels_tune20 = []

    d = {'0':0,'1':0,'2':0,'3':0,'4':0,'5':0,'6':0,'7':0,'8':0,'9':0}

    for i in items:
    
        label = label_func(i)
        if d[label] == 0 or d[label] == 1:
            items_tune20.append(i)
            labels_tune20.append(label)
            d[label] += 1
            raw.append(i)

    dls_val = ImageDataLoaders.from_lists(path, items_tune20, labels_tune20,bs=20,
                                  valid_pct=0.0,num_workers=2*(device=='cuda'),device=device,
                                      img_cls=PILImageBW)

    d = {'items_tune20':items_tune20,'labels_tune20':labels_tune20,'raw':raw,'dls_val':dls_val}
    
    return d


In [ ]:
#Test dls_train, dls_val, dls_test and that function `tune_set` is working 

test_eq(seed,42)
test_eq(ts,10000)

x,y = dls_train.one_batch()
test_eq(x.shape, [bs, 1, 28, 28])

test_eq(len(dls_train.train_ds),ts)
test_eq(len(dls_test.train_ds),ts)

test_eq(len(dls_test.train)*bs_test,len(dls_test.train_ds))

_tune_set = tune_set(items_tune,seed=seed+1)
items_tune20,labels_tune20,dls_val,raw = _tune_set['items_tune20'],_tune_set['labels_tune20'], _tune_set['dls_val'],_tune_set['raw']


my_dict = {i:labels_tune20.count(i) for i in labels_tune20}
for i in my_dict.values():
    test_eq(i,2)

#test size
test_eq(len(dls_val.train),1)
test_eq(len(dls_val.train_ds),20)

#test random seed (please note that this depends on random seed chosen above to sort initially, i.e. seed=42)
_raw = tune_set(items_tune,seed=1)['raw']
test_eq(_raw[0].name,'28316.png')

_raw = tune_set(items_tune,seed=10)['raw']
test_eq(_raw[11].name,'11658.png')


Test the random seed:

Patch in definition of loss function (BT for now):

In [16]:
class Max_Corr(nn.Module):
    def __init__(self,qs,default,normal=True,std=0.01,s_unif=0.01):
        super().__init__()
        self.qs=qs
        self.std=std
        self.s_unif=s_unif
        self.normal=normal
        self.default=default

        if not self.default:
        
            self.fc1 = nn.utils.skip_init(nn.Linear, qs, qs)
            self.fc11 = nn.utils.skip_init(nn.Linear, qs, qs)
            self.fc2 = nn.utils.skip_init(nn.Linear, qs, qs)
            self.fc22 = nn.utils.skip_init(nn.Linear, qs, qs)

        else:

            assert False

            self.fc1 = nn.Linear(qs, qs)
            self.fc11 = nn.Linear(qs, qs)
            self.fc2 = nn.Linear(qs, qs)
            self.fc22 = nn.Linear(qs, qs)

    
        self.relu = nn.ReLU()

        if self.normal and not self.default:


            torch.nn.init.normal_(self.fc1.weight, std=self.std)
            torch.nn.init.normal_(self.fc11.weight, std=self.std)
            torch.nn.init.normal_(self.fc2.weight, std=self.std)
            torch.nn.init.normal_(self.fc22.weight, std=self.std)

            torch.nn.init.normal_(self.fc1.bias, std=self.std)
            torch.nn.init.normal_(self.fc11.bias, std=self.std)
            torch.nn.init.normal_(self.fc2.bias, std=self.std)
            torch.nn.init.normal_(self.fc22.bias, std=self.std)

        elif not self.normal and not self.default:

            assert False

            torch.nn.init.uniform_(self.fc1.weight, a=-self.s_unif,b=self.s_unif)
            torch.nn.init.uniform_(self.fc11.weight, a=-self.s_unif,b=self.s_unif)
            torch.nn.init.uniform_(self.fc2.weight, a=-self.s_unif,b=self.s_unif)
            torch.nn.init.uniform_(self.fc22.weight, a=-self.s_unif,b=self.s_unif)

            torch.nn.init.uniform_(self.fc1.bias, a=-self.s_unif,b=self.s_unif)
            torch.nn.init.uniform_(self.fc11.bias, a=-self.s_unif,b=self.s_unif)
            torch.nn.init.uniform_(self.fc2.bias, a=-self.s_unif,b=self.s_unif)
            torch.nn.init.uniform_(self.fc22.bias, a=-self.s_unif,b=self.s_unif)


        self.m1 = nn.Sequential(self.fc1,self.relu,self.fc11)
        self.m2 = nn.Sequential(self.fc2,self.relu,self.fc22)

    def forward(self,x,y):
        return self.m1(x),self.m2(y)
        
#| export
class Cdiff_Sup:
    
    def __init__(self,I,qs,inner_steps,normal,std,s_unif,default,indep=True):
        
        self.I=I
        self.inner_steps=inner_steps
        self.indep=indep

        self.max_corr = Max_Corr(qs=qs,default=default,normal=normal,std=std,s_unif=s_unif)
        if default_device().type == 'cuda':
            self.max_corr.cuda()
        
    def inner_step(self,z1norm,z2norm):
    
        max_corr=self.max_corr
        inner_steps=self.inner_steps
        z1norm=z1norm.detach()
        z2norm=z2norm.detach()

        optimizer = torch.optim.Adam(list(max_corr.parameters()),lr=0.001)
        #optimizer = torch.optim.SGD(list(max_corr.parameters()),lr=0.01)
        
        for i in range(inner_steps):

            z1norm_2,z2norm_2=max_corr(z1norm,z2norm)
            cdiff_2 = C_z1z2(z1norm=z1norm,z1norm_2=z1norm_2,z2norm=z2norm,z2norm_2=z2norm_2,indep=self.indep)
            inner_loss=-1*(cdiff_2*(1-self.I)).mean()
            optimizer.zero_grad()
            inner_loss.backward()
            optimizer.step()
        
        for p in max_corr.parameters():
            p.requires_grad=False
            
        return max_corr
    
    def __call__(self,z1norm,z2norm):
        
            max_corr =  self.inner_step(z1norm,z2norm)
            z1norm_2,z2norm_2 = max_corr(z1norm,z2norm)
            cdiff_sup = C_z1z2(z1norm=z1norm,z1norm_2=z1norm_2,z2norm=z2norm,z2norm_2=z2norm_2,indep=self.indep)
            
            return cdiff_sup

In [7]:
class AttributeDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


HP = AttributeDict(t=0.5, #convexity level
               indep_rand=True,indep_sup=False,enc_rand=False,enc_sup=True, 
               inner_steps=15,act1='relu',act2='sigmoid', #belong to sup term
               K=10,phase_std=0.2,amp_std=0.2,off_std=1)


def lf_rbt_enc(pred,I,lmb,
               hp,
               ):

    pred_enc = pred[0]
    pred = pred[1]
    
    bs,nf = pred.size(0)//2,pred.size(1)

    #All standard, from BT
    z1, z2 = pred[:bs],pred[bs:] #so z1 is bs*projection_size, likewise for z2
    z1norm = (z1 - z1.mean(0)) / z1.std(0, unbiased=False)
    z2norm = (z2 - z2.mean(0)) / z2.std(0, unbiased=False)
    
    z1_enc, z2_enc = pred_enc[:bs],pred_enc[bs:] #so z1 is bs*projection_size, likewise for z2
    z1norm_enc = (z1_enc - z1_enc.mean(0)) / (z1_enc.std(0, unbiased=False) + 1e-7)
    z2norm_enc = (z2_enc - z2_enc.mean(0)) / (z2_enc.std(0, unbiased=False) + 1e-7)

    nf_enc = z1_enc.shape[1]
    I_enc = torch.eye(nf_enc).to(device)

    if hp.enc_rr:
        z1norm_sup = z1norm_enc
        z2norm_sup = z2norm_enc
        qs_sup = nf_enc
        I_sup = I_enc

    else:
        z1norm_sup = z1norm
        z2norm_sup = z2norm
        qs_sup = nf
        I_sup = I
    
    if hp.enc_inv:
        assert False
        C = (z1norm_enc.T @ z2norm_enc) / bs
        I_inv = I_enc
        cdiff = (C - I_inv)**2

    else:
        C = (z1norm.T @ z2norm) / bs 
        I_inv = I
        cdiff = (C - I)**2


    CdiffSup = Cdiff_Sup(I=I_sup,qs=qs_sup,inner_steps=hp.inner_steps,default=hp.default,indep=hp.indep_sup,std=hp.std,s_unif=hp.s_unif,normal=hp.normal)
    cdiff_sup = CdiffSup(z1norm_sup,z2norm_sup) #same shape as cdiff)

    rr=hp.t*(cdiff_sup*(1-I_sup)).sum()

    #Uncomment
    loss = (cdiff*I_inv).sum() + rr  #sum of redundancy reduction term and invariance term
    
    #loss = (cdiff_2*I_inv).sum() + lmb*(cdiff*(1-I)).sum()

    #loss = (cdiff*I).sum() + lmb*(cdiff*(1-I)).sum() #standard BT
    
    torch.cuda.empty_cache()
    return loss

In [ ]:
# #Using RBT_enc
@patch
def lf(self:BarlowTwins, pred,*yb): return lf_rbt_enc(pred,I=self.I,lmb=self.lmb,hp=HP)

We also patch `before_epoch` so that we are training linear classifier:
    

Setup for linear evaluation:

In [8]:
aug_pipelines_val=[get_linear_batch_augs(size=28,stats=mnist_stats,resize=True,resize_scale=(0.4, 1.0))]

# main_linear_eval = Main_Linear_Eval(size=size,n_in=n_in,numfit=numfit,indim=1024, #size,n_in=3 (color channels),number of epochs to fit linear, and output dimension of encoder
#                     dls_val=dls_val,dls_test=dls_test, #dls for training linear and evaluating linear
#                     stats=mnist_stats,
#                     aug_pipelines_val=aug_pipelines_val, #aug_pipeline for training 
#                     encoder=None#encoder
#                                 )

Optional: Show linear augmentations:

In [ ]:
show_linear_batch(dls=dls_val,n_in=n_in,n=2,aug=aug_pipelines_val,print_augs=True)

Patch linear evaluation into `after_epoch:`

Define encoder and model; Define augmentation pipelines; Define learner.  

In [9]:
fastai_encoder = create_fastai_encoder(xresnet18(),pretrained=False,n_in=n_in)

#If we are using a different model, this call will just look like `create_rat_model(...)`
model = create_p3barlow_twins_model(fastai_encoder, hidden_size=ps,projection_size=ps,nlayers=3)

aug_pipelines_1 = get_barlow_twins_aug_pipelines(size=28,blur_r=(0.5,2),
                    rotate=True,jitter=False,noise=True,bw=False,blur=True,solar=True, #Whether to use aug or not
                    resize_scale=(0.5, 1.0),resize_ratio=(3/4, 4/3),noise_std=0.05, rotate_deg=25,blur_s=11,sol_t=0.025,sol_a=0.025, #hps of augs
                    flip_p=0.1, rotate_p=0.5,noise_p=0.5, jitter_p=0.3, bw_p=0.3, blur_p=0.5,sol_p=0.1, #prob of performing aug
                    same_on_batch=False,stats=mnist_stats, cuda=(device=='cuda'))

aug_pipelines_2 = get_barlow_twins_aug_pipelines(size=28,blur_r=(0.5,2),
                    rotate=True,jitter=False,noise=True,bw=False,blur=True,solar=True, #Whether to use aug or not
                    resize_scale=(0.5, 1.0),resize_ratio=(3/4, 4/3),noise_std=0.05, rotate_deg=25,blur_s=11,sol_t=0.025,sol_a=0.025, #hps of augs
                    flip_p=0.1, rotate_p=0.5,noise_p=0.5, jitter_p=0.3, bw_p=0.3, blur_p=0.5,sol_p=0.1, #prob of performing aug
                    same_on_batch=False,stats=mnist_stats, cuda=(device=='cuda'))


#aug_pipelines = get_barlow_twins_aug_pipelines(size=28, rotate=True,flip_p=0,resize_scale=(0.7,1), jitter=False, bw=False,blur=True,blur_p=0.5,blur_s=8, stats=None, cuda=True)

# noise=True,rotate=True,jitter=True,bw=True,blur=True,solar=True, #Whether to use  given aug or not
# resize_scale=(0.08, 1.0),resize_ratio=(3/4, 4/3),noise_std=0.025, rotate_deg=30,jitter_s=.6,blur_s=(4,32),s1=None,sol_t=0.05,sol_a=0.05, #hps of diff augs
# flip_p=0.5, rotate_p=0.3,noise_p=0.2, jitter_p=0.3, bw_p=0.3, blur_p=0.3,sol_p=0.1, #prob of performing aug
# same_on_batch=False,stats=imagenet_stats,cuda=default_device().type == 'cuda',xtra_tfms=[])

aug_pipelines = [aug_pipelines_1,aug_pipelines_2]

#If we are using a different `callback` to `BarlowTwins` then we can simply replace `BarlowTwins` with 
#e.g. `BarlowTriplets`. We can define in base_model and just import with no issues. 
learn = Learner(dls_train,model, cbs=[BarlowTwins(aug_pipelines,n_in=n_in,lmb=1/ps,print_augs=False)])
#Set path of learn correctly
#learn.path = Path('/content/drive/MyDrive/')

Step 3): (Optional): View the augmentations:

In [ ]:
show_bt_batch(dls=dls_train,n_in=n_in,aug=aug_pipelines,n=2,print_augs=True)

Step 4): Fit the learner:

All we have changed from "best so far" on current run is have _enc on both terms (sup and rand)
instead of just on the _proj term

In [10]:
def sample_hp():
    "Sample hps for training bt -- need number of inner_steps and a bunch of other stuff"

    # t = random.choice([0.01])
    # std = random.choice([0.005,0.01,0.02])
    # s_unif = random.choice([0.005,0.01,0.02])
    # normal = random.choice([True,False])

    return AttributeDict({'t':0.01,
    'normal':True,
     'std':0.02,
     's_unif':'foo',
    'inner_steps':0,
   'indep_sup': False, #This will always be false for now
   'enc_rr': False, #use encoder for rr term (else use projector)
   'enc_inv':False}) #use encoder for invariance term (else use projector)



In [11]:
def train_rbt(hp):
        
    @patch
    def lf(self:BarlowTwins, pred,*yb): return lf_rbt_enc(pred,I=self.I,lmb=self.lmb,hp=hp) #pass them to loss function
  
    #train
    fastai_encoder = create_fastai_encoder(xresnet18(),pretrained=False,n_in=n_in)
    model = create_p2barlow_twins_model(fastai_encoder, hidden_size=ps,projection_size=ps,nlayers=3)
    learn = Learner(dls_train,model, cbs=[BarlowTwins(aug_pipelines,n_in=n_in,lmb=1/ps,print_augs=False)])
    learn.fit(300)

    return fastai_encoder


In [12]:
#Tune run for given hps

def tune_valid(fastai_encoder):
    fastai_encoder.eval() #eval mode
    fastai_encoder = grad_on(fastai_encoder,on=False) #turn off gradients
    acc_dict={}
    for i in range(10):
        dls_val = tune_set(items_tune,seed=seed+i)['dls_val']

        main_linear_eval = Main_Linear_Eval(size=size,n_in=n_in,numfit=numfit,indim=1024, #size,n_in=3 (color channels),number of epochs to fit linear, and output dimension of encoder
                                dls_val=dls_val,dls_test=dls_test, #dls for training linear and evaluating linear
                                stats=mnist_stats,
                                aug_pipelines_val=aug_pipelines_val, #aug_pipeline for training 
                                encoder=fastai_encoder #encoder
                                            )
        #main_linear_eval.encoder = self.encoder (usage within BT training)
        acc=main_linear_eval()
        acc_dict[i]=acc
        
    acc_dict['mean'] = (sum(list(acc_dict.values()))/len(acc_dict.values()))
    print((sum(list(acc_dict.values()))/len(acc_dict.values())))
    
    return acc_dict

Plan:
 - Grid search for best values for normal (using epochs=50),10 tune sets. Then run for 300 epochs, on 10 tune sets.
 - Grid search for best values for uniform (using epochs=50). Then run for 300 epochs, on 10 tune sets.

In [17]:
result={} #store the hp dict in here and the acc_dict results for the given hp

t = [0.01]
std = [0.01]
hp_vals = [t,std]
hp_vals = itertools.product(*hp_vals)

for i,hp_val in enumerate(hp_vals):

    t=hp_val[0]
    std=hp_val[1]

    hp = AttributeDict({'t':t,
                        'std':std,
                        'default':False,
                        'normal':True,
                        's_unif':'foo',
                        'inner_steps':0,
                        'indep_sup': False, #This will always be false for now
                        'enc_rr': False, #use encoder for rr term (else use projector)
                        'enc_inv':False
                        }) #use encoder for in


    fastai_encoder = train_rbt(hp) #train encoder
    acc_dict = tune_valid(fastai_encoder) #train head and evaluate
    result[i] = {'hp':hp,'acc_dict':acc_dict} #so we are looking for the hp that has highest mean value for acc_dict
    print(result[i]['acc_dict']) 
    print(result[i]['acc_dict']['mean'])
    print(result[i]['hp'])
    print('\n')
    print('done')
    
print(acc_dict)
result
print('Running with pytorch init defaults')

epoch,train_loss,valid_loss,time
0,42.728580,None,00:14
1,27.317244,None,00:14
2,22.241501,None,00:14
3,20.192707,None,00:14
4,16.701487,None,00:14
5,15.743664,None,00:14
6,14.905575,None,00:14
7,14.803210,None,00:14
8,13.044523,None,00:14
9,13.599191,None,00:14


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


epoch,train_loss,valid_loss,time
0,2.465875,None,00:00
1,2.301230,None,00:00
2,2.142746,None,00:00
3,1.985703,None,00:00
4,1.841401,None,00:00
5,1.709254,None,00:00
6,1.585414,None,00:00
7,1.471689,None,00:00
8,1.366621,None,00:00
9,1.271324,None,00:00


epoch,train_loss,valid_loss,time
0,2.292872,None,00:00
1,2.140024,None,00:00
2,2.001236,None,00:00
3,1.853673,None,00:00
4,1.761091,None,00:00
5,1.636209,None,00:00
6,1.516617,None,00:00
7,1.410525,None,00:00
8,1.317857,None,00:00
9,1.235996,None,00:00


epoch,train_loss,valid_loss,time
0,2.351097,None,00:00
1,2.185640,None,00:00
2,2.031747,None,00:00
3,1.889467,None,00:00
4,1.758046,None,00:00
5,1.661811,None,00:00
6,1.556837,None,00:00
7,1.451499,None,00:00
8,1.355361,None,00:00
9,1.268854,None,00:00


...

epoch,train_loss,valid_loss,time
0,2.394684,None,00:00
1,2.245935,None,00:00
2,2.099190,None,00:00
3,1.959248,None,00:00
4,1.836962,None,00:00
5,1.717331,None,00:00
6,1.607602,None,00:00
7,1.547235,None,00:00
8,1.450702,None,00:00
9,1.364126,None,00:00


epoch,train_loss,valid_loss,time
0,2.264420,None,00:00
1,2.098402,None,00:00
2,1.965294,None,00:00
3,1.856934,None,00:00
4,1.726645,None,00:00
5,1.603683,None,00:00
6,1.491441,None,00:00
7,1.391993,None,00:00
8,1.307788,None,00:00
9,1.230835,None,00:00


epoch,train_loss,valid_loss,time
0,2.245590,None,00:00
1,2.084717,None,00:00
2,1.933616,None,00:00
3,1.800917,None,00:00
4,1.668919,None,00:00
5,1.549580,None,00:00
6,1.440210,None,00:00
7,1.340446,None,00:00
8,1.255731,None,00:00
9,1.179122,None,00:00


epoch,train_loss,valid_loss,time
0,2.364996,None,00:00
1,2.215430,None,00:00
2,2.060507,None,00:00
3,1.913110,None,00:00
4,1.780401,None,00:00
5,1.654686,None,00:00
6,1.539249,None,00:00
7,1.429083,None,00:00
8,1.329457,None,00:00
9,1.236856,None,00:00


0.8454500019550324
{0: 0.8580999970436096, 1: 0.7968999743461609, 2: 0.8806999921798706, 3: 0.7922999858856201, 4: 0.8841000199317932, 5: 0.8112000226974487, 6: 0.8515999913215637, 7: 0.9146000146865845, 8: 0.8438000082969666, 9: 0.8212000131607056, 'mean': 0.8454500019550324}
0.8454500019550324
{'t': 0.01, 'std': 0.01, 'default': False, 'normal': True, 's_unif': 'foo', 'inner_steps': 0, 'indep_sup': False, 'enc_rr': False, 'enc_inv': False}


done
{0: 0.8580999970436096, 1: 0.7968999743461609, 2: 0.8806999921798706, 3: 0.7922999858856201, 4: 0.8841000199317932, 5: 0.8112000226974487, 6: 0.8515999913215637, 7: 0.9146000146865845, 8: 0.8438000082969666, 9: 0.8212000131607056, 'mean': 0.8454500019550324}
Running with pytorch init defaults


BT baseline:
- 300 epochs, ps=1024, seed=20:


In [ ]:
BT={0: 0.8694000244140625, 1: 0.845300018787384, 2: 0.9122999906539917, 3: 0.838100016117096, 4: 0.9002000093460083, 5: 0.8216999769210815, 6: 0.9039000272750854, 7: 0.9164999723434448, 8: 0.8743000030517578, 9: 0.8971999883651733, 'mean': 0.8778900027275085}


Results using same setup at BT baseline:

In [ ]:
#300 epochs

#pytorch defaults
{0: 0.8784000277519226, 1: 0.8495000004768372, 2: 0.9190000295639038, 3: 0.8008999824523926, 4: 0.8709999918937683, 5: 0.8210999965667725, 6: 0.8664000034332275, 7: 0.9254000186920166, 8: 0.8367999792098999, 9: 0.8838000297546387, 'mean': 0.8652300059795379}
0.8652300059795379
{'t': 0.02, 'std': 'foo', 'default': True, 'normal': 'foo', 's_unif': 'foo', 'inner_steps': 0, 'indep_sup': False, 'enc_rr': False, 'enc_inv': False}

{0: 0.8881999850273132, 1: 0.8543999791145325, 2: 0.913100004196167, 3: 0.8033000230789185, 4: 0.8802000284194946, 5: 0.8319000005722046, 6: 0.8779000043869019, 7: 0.9261999726295471, 8: 0.8547999858856201, 9: 0.8754000067710876, 'mean': 0.8705399990081787}
0.8705399990081787
{'t': 0.01, 'std': 'foo', 'default': True, 'normal': 'foo', 's_unif': 'foo', 'inner_steps': 0, 'indep_sup': False, 'enc_rr': False, 'enc_inv': False}


{0: 0.8809000253677368, 1: 0.8403000235557556, 2: 0.9032999873161316, 3: 0.8256999850273132, 4: 0.9118000268936157, 5: 0.840499997138977, 6: 0.9081000089645386, 7: 0.9283000230789185, 8: 0.838699996471405, 9: 0.8639000058174133, 'mean': 0.8741500079631805}
0.8741500079631805
{'t': 0.005, 'std': 'foo', 'default': True, 'normal': 'foo', 's_unif': 'foo', 'inner_steps': 0, 'indep_sup': False, 'enc_rr': False, 'enc_inv': False}

0.8759699940681458
{0: 0.895799994468689, 1: 0.855400025844574, 2: 0.9217000007629395, 3: 0.8044999837875366, 4: 0.9179999828338623, 5: 0.8486999869346619, 6: 0.8934000134468079, 7: 0.9243999719619751, 8: 0.8393999934196472, 9: 0.8583999872207642, 'mean': 0.8759699940681458}
0.8759699940681458
{'t': 0.0025, 'std': 'foo', 'default': True, 'normal': 'foo', 's_unif': 'foo', 'inner_steps': 0, 'indep_sup': False, 'enc_rr': False, 'enc_inv': False}

{0: 0.868399977684021, 1: 0.8235999941825867, 2: 0.8824999928474426, 3: 0.8051999807357788, 4: 0.8974000215530396, 5: 0.8424999713897705, 6: 0.8565999865531921, 7: 0.9259999990463257, 8: 0.8604000210762024, 9: 0.864799976348877, 'mean': 0.8627399921417236}
0.8627399921417236
{'t': 0.00125, 'std': 'foo', 'default': True, 'normal': 'foo', 's_unif': 'foo', 'inner_steps': 0, 'indep_sup': False, 'enc_rr': False, 'enc_inv': False}

{0: 0.8580999970436096, 1: 0.7968999743461609, 2: 0.8806999921798706, 3: 0.7922999858856201, 4: 0.8841000199317932, 5: 0.8112000226974487, 6: 0.8515999913215637, 7: 0.9146000146865845, 8: 0.8438000082969666, 9: 0.8212000131607056, 'mean': 0.8454500019550324}
0.8454500019550324
{'t': 0.01, 'std': 0.01, 'default': False, 'normal': True, 's_unif': 'foo', 'inner_steps': 0, 'indep_sup': False, 'enc_rr': False, 'enc_inv': False}


#########
#Uniform:
#########
{'t': 0.02,'std': 'foo','normal': False,'s_unif': 0.04,'inner_steps': 0,'indep_sup': False,'enc_rr': False,'enc_inv': False},
{0: 0.8343999981880188,
 1: 0.7953000068664551,
 2: 0.8870000243186951,
 3: 0.788100004196167,
 4: 0.8235999941825867,
 5: 0.8202999830245972,
 6: 0.8514999747276306,
 7: 0.8966000080108643,
 8: 0.7949000000953674,
 9: 0.8603000044822693,
 'mean': 0.8351999998092652}

#########
{0: {'hp': {'t': 0.02,
   'std': 'foo',
   'normal': False,
   's_unif': 0.02,
   'inner_steps': 0,
   'indep_sup': False,
   'enc_rr': False,
   'enc_inv': False},
  'acc_dict': {0: 0.878600001335144,
   1: 0.8282999992370605,
   2: 0.9061999917030334,
   3: 0.8288000226020813,
   4: 0.9042999744415283,
   5: 0.840399980545044,
   6: 0.8952000141143799,
   7: 0.9232000112533569,
   8: 0.8501999974250793,
   9: 0.8593999743461609,
   'mean': 0.8714599967002868}}}
#########

{0: {'hp': {'t': 0.02,
   'std': 'foo',
   'normal': False,
   's_unif': 0.01,
   'inner_steps': 0,
   'indep_sup': False,
   'enc_rr': False,
   'enc_inv': False},
  'acc_dict': {0: 0.7552000284194946,
   1: 0.6628999710083008,
   2: 0.7287999987602234,
   3: 0.6334999799728394,
   4: 0.7738999724388123,
   5: 0.7195000052452087,
   6: 0.6976000070571899,
   7: 0.8287000060081482,
   8: 0.7653999924659729,
   9: 0.6791999936103821,
   'mean': 0.7244699954986572}}}
#########

#Normal:
{0: {'hp': {'t': 0.02,
   'std': 0.02,
   'normal': True,
   's_unif': 'foo',
   'inner_steps': 0,
   'indep_sup': False,
   'enc_rr': False,
   'enc_inv': False},
  'acc_dict': {0: 0.8841999769210815,
   1: 0.8210999965667725,
   2: 0.9014999866485596,
   3: 0.7718999981880188,
   4: 0.8485000133514404,
   5: 0.8123000264167786,
   6: 0.8804000020027161,
   7: 0.9193000197410583,
   8: 0.8105999827384949,
   9: 0.8723999857902527,
   'mean': 0.8522199988365173}}}
#########

{0: {'hp': {'t': 0.02,
   'std': 0.01,
   'normal': True,
   's_unif': 'foo',
   'inner_steps': 0,
   'indep_sup': False,
   'enc_rr': False,
   'enc_inv': False},
  'acc_dict': {0: 0.8924999833106995,
   1: 0.8560000061988831,
   2: 0.9085000157356262,
   3: 0.8210999965667725,
   4: 0.9196000099182129,
   5: 0.8490999937057495,
   6: 0.8934999704360962,
   7: 0.9207000136375427,
   8: 0.8543999791145325,
   9: 0.8589000105857849,
   'mean': 0.87742999792099}}}

#########

   {'hp': {'t': 0.02,
   'std': 0.005,
   'normal': True,
   's_unif': 'foo',
   'inner_steps': 0,
   'indep_sup': False,
   'enc_rr': False,
   'enc_inv': False},
  'acc_dict': {0: 0.6044999957084656,
   1: 0.5041999816894531,
   2: 0.5716000199317932,
   3: 0.49070000648498535,
   4: 0.5263000130653381,
   5: 0.526199996471405,
   6: 0.5309000015258789,
   7: 0.6097000241279602,
   8: 0.5480999946594238,
   9: 0.5396000146865845,
   'mean': 0.5451800048351287}}